# Importar librerias

In [1]:
import pandas as pd
import numpy as np
import emoji
import re
import spacy
import nltk

from nltk.corpus import stopwords
from spacy.lang.es.stop_words import STOP_WORDS

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from gensim.models import Word2Vec

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical

# Cargar datos de entrenamiento

In [2]:
path = '../notebooks/data/sentiment_analysis_dataset.csv'
data = pd.read_csv(path)
data.head()

,user,text,date,emotion,sentiment
0,@erreborda,termine bien abrumado después de hoy,"Jan 6, 2024 · 2:53 AM UTC",overwhelmed,scared
1,@shpiderduck,me siento abrumado❤,"Jan 6, 2024 · 2:35 AM UTC",overwhelmed,scared
2,@Alex_R_art,Me siento un poco abrumado por la cantidad de ...,"Jan 6, 2024 · 12:20 AM UTC",overwhelmed,scared
3,@anggelinaa97,Salvador la única persona que no la ha abrumad...,"Jan 5, 2024 · 10:38 PM UTC",overwhelmed,scared
4,@diegoreyesvqz,Denme un helado o algo que ando full abrumado.,"Jan 5, 2024 · 8:38 PM UTC",overwhelmed,scared


In [3]:
data['emotion'].value_counts().count()


20

In [4]:
data['sentiment'].value_counts()

sentiment
peaceful    660
mad         530
powerful    420
sad         360
joyful      350
scared      270
Name: count, dtype: int64

# Preprocesamiento (Limpieza, Tokenización, Stopwords y Lematización)

In [5]:
# Traducción de las etiquetas a español
translation = {
    'joyful': 'Alegre',
    'daring': 'Osado',
    'optimistic': 'Optimista',
    'playful': 'Jugueton',
    'powerful': 'Poderoso',
    'surprised': 'Sorprendido',
    'successful': 'Exitoso',
    'confident': 'Confiado',
    'peaceful': 'Tranquilo',
    'secure': 'Seguro',
    'thankful': 'Agradecido',
    'loving': 'Amoroso',
    'relaxed': 'Relajado',
    'responsive': 'Sensible',
    'sad': 'Triste',
    'sleepy': 'Adormilado',
    'isolated': 'Aislado',
    'stupid': 'Estupido',
    'mad': 'Histerico',
    'distant': 'Distante',
    'frustrated': 'Frustrado',
    'irritated': 'Irritado',
    'jealous': 'Celoso',
    'scared': 'Asustado',
    'embarrassed': 'Avergonzado',
    'overwhelmed': 'Agobiado',
}

In [6]:
data[['emotion', 'sentiment']] = data[['emotion', 'sentiment']].replace(to_replace = translation)
data.head()

,user,text,date,emotion,sentiment
0,@erreborda,termine bien abrumado después de hoy,"Jan 6, 2024 · 2:53 AM UTC",Agobiado,Asustado
1,@shpiderduck,me siento abrumado❤,"Jan 6, 2024 · 2:35 AM UTC",Agobiado,Asustado
2,@Alex_R_art,Me siento un poco abrumado por la cantidad de ...,"Jan 6, 2024 · 12:20 AM UTC",Agobiado,Asustado
3,@anggelinaa97,Salvador la única persona que no la ha abrumad...,"Jan 5, 2024 · 10:38 PM UTC",Agobiado,Asustado
4,@diegoreyesvqz,Denme un helado o algo que ando full abrumado.,"Jan 5, 2024 · 8:38 PM UTC",Agobiado,Asustado


In [7]:
# target_map ={
#     'Alegre': 1, # Sentimiento 1
#     'Osado': 1,
#     'Optimista': 2,
#     'Jugueton': 3,
#     'Poderoso': 2, # Sentimiento 2
#     'Sorprendido': 4,
#     'Exitoso': 5, 
#     'Confiado': 6, 
#     'Tranquilo': 3, # Sentimiento 3
#     'Seguro': 7,
#     'Agradecido': 8,
#     'Amoroso': 9,
#     'Relajado': 10,
#     'Sensible': 11,
#     'Triste': 4, # Sentimiento 4
#     'Adormilado': 12,
#     'Aislado': 13,
#     'Estupido': 14,
#     'Histerico': 5, # Sentimiento 5
#     'Distante': 15,
#     'Frustrado': 16,
#     'Irritado': 17,
#     'Celoso': 18,
#     'Asustado': 6, # Sentimiento 6
#     'Avergonzado': 19,
#     'Agobiado': 20,
# }

In [8]:
# emotions = ['Osado', 'Optimista', 'Jugueton', 'Sorprendido', 'Exitoso', 'Confiado', 'Seguro', 'Agradecido', 'Amoroso', 'Relajado', 'Sensible', 
#            'Adormilado', 'Aislado', 'Histerico', 'Distante', 'Frustrado', 'Irritado', 'Celoso', 'Avergonzado', 'Agobiado']

In [9]:
# reversed_map = {value: sentiment for sentiment, value in target_map.items()}

In [10]:
sentiments_map = {
    'Osado': 'Felicidad',
    'Optimista': 'Felicidad',
    'Jugueton': 'Felicidad',
    'Sorprendido': 'Empoderado',
    'Exitoso': 'Empoderado',
    'Confiado': 'Empoderado',
    'Seguro': 'Paz',
    'Agradecido': 'Paz',
    'Amoroso': 'Paz',
    'Relajado': 'Paz',
    'Sensible': 'Paz',
    'Adormilado': 'Tristeza',
    'Aislado': 'Tristeza',
    'Histerico': 'Tristeza',
    'Distante': 'Furia',
    'Frustrado': 'Furia',
    'Irritado': 'Furia',
    'Celoso': 'Furia',
    'Avergonzado': 'Miedo',
    'Agobiado': 'Miedo',
}

In [11]:
# data['emotion'] = data['emotion'].map(target_map)
# data.head()

In [12]:
df = data[['text', 'emotion']].copy()
df.rename(columns = {'emotion': 'target'}, inplace = True)

In [13]:
nltk.download('stopwords')
nltk.download('punkt')

# Cargar el modelo de spaCy para español
nlp = spacy.load("es_core_news_sm")

# Definir stopwords en español
stop_words = set(stopwords.words('spanish')) | STOP_WORDS

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\WilmarAl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\WilmarAl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
def preprocess_text(text):
    text = emoji.replace_emoji(text, replace='')
    text = text.lower()
    text = text.replace('á','a').replace('é','e')
    text = text.replace('í','i').replace('ó','o')
    text = text.replace('ú','u').replace('$','')
    text = text.replace('—','').replace('-',' ')
    text = text.replace('%','').replace('&','')
    text = text.replace('\n',' ').replace('\t',' ')
    text = text.replace("'","").replace('"','')
    text = text.replace(',','').replace('.','')
    text = text.replace(';','').replace(':','')
    text = re.sub(r'@[A-Za-z0-9_]+', '', text)  # Eliminar menciones
    text = re.sub(r'#[A-Za-z0-9_]+', '', text)  # Eliminar hashtags
    text = re.sub(r'http\S+|www\.\S+', '', text)  # Eliminar URLs
    text = re.sub(r'[^a-zA-ZáéíóúÁÉÍÓÚñÑ ]', '', text)  # Eliminar caracteres especiales y emojis

    doc = nlp(text)
    
    # Eliminar stopwords y lematizar
    palabras_procesadas = [token.lemma_ for token in doc if token.text.lower() not in stop_words and not token.is_punct]
    
    return " ".join(palabras_procesadas)

def text_to_vector(tokens, model):
    # Obtener el vector promedio de todas las palabras en el texto
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)  # Promedio de los vectores
    else:
        return np.zeros(model.vector_size)  # Si no hay palabras conocidas, devolver un vector de ceros

In [15]:
df['tokens'] = df['text'].apply(preprocess_text)

In [16]:
df.head()

,text,target,tokens
0,termine bien abrumado después de hoy,Agobiado,terminar abrumado
1,me siento abrumado❤,Agobiado,sentir abrumado
2,Me siento un poco abrumado por la cantidad de ...,Agobiado,sentir abrumado cantidad cosa querer dibujar j...
3,Salvador la única persona que no la ha abrumad...,Agobiado,salvador unico persona abrumar versión
4,Denme un helado o algo que ando full abrumado.,Agobiado,denmir helado ar full abrumado


In [ ]:
word2vec_model = Word2Vec(sentences = df['tokens'], vector_size=5, window=5, min_count=1, workers=2)

# Vectorización

In [18]:
# Aplicar la vectorización a cada fila del dataframe
df['vector'] = df['tokens'].apply(lambda x: text_to_vector(x, word2vec_model))

In [19]:
df.sample(2)

,text,target,tokens,vector
1769,"""¿Ahora es cuando te rompo la columna para que...",Seguro,rompo columna chupes alzo ceja sonrisa tono ...,"[-0.22160594, -0.14141804, 0.37716672, -0.3684..."
758,viernes 23:58 me puse a jugar al solitario se...,Distante,viernes poner jugar solitario venir nochon,"[-0.273233, -0.11118002, 0.37799674, -0.392835..."


In [20]:
X = np.array(df['vector'].tolist())  # Matriz de características (vectores)
y = df['target']  # Etiquetas (emotion)

In [21]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)  # Convertir etiquetas a números
y_categorical = to_categorical(y_encoded)   # Convertir a one-hot encoding

# Train Test Split

In [22]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42 )

# Entrenamiento y desempeño de los modelos

In [23]:
# 
model_ann = keras.Sequential([
    layers.Dense(64, activation='relu', input_dim = 10),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(20, activation='softmax')
])

c:\Users\WilmarAl\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [24]:
# Compilar el modelo
model_ann.compile(
    optimizer='adam',  # Optimizador Adam
    loss='categorical_crossentropy',  # Función de pérdida para clasificación multiclase
    metrics=['accuracy']  # Métrica de evaluación
)

In [25]:
# Entrenar la red neuronal
history = model_ann.fit(X_train, Y_train, epochs=20, batch_size=16, validation_data = (X_test, Y_test))

Epoch 1/20
130/130 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0438 - loss: 3.0146 - val_accuracy: 0.0405 - val_loss: 2.9974
Epoch 2/20
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0625 - loss: 2.9976 - val_accuracy: 0.0444 - val_loss: 2.9962
Epoch 3/20
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0586 - loss: 2.9938 - val_accuracy: 0.0425 - val_loss: 2.9974
Epoch 4/20
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0470 - loss: 2.9924 - val_accuracy: 0.0541 - val_loss: 2.9965
Epoch 5/20
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0663 - loss: 2.9918 - val_accuracy: 0.0598 - val_loss: 2.9962
Epoch 6/20
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0572 - loss: 2.9904 - val_accuracy: 0.0598 - val_loss: 2.9971
Epoch 7/20
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0559 - loss: 2.9913 - val_accuracy: 0.0618 - val_loss: 2.9973
Epoch 8/20
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0531 - loss: 2.9905 - val_accuracy: 0.

In [26]:
# 10. Evaluar el modelo
y_pred = model_ann.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convertir one-hot encoding a clases
y_test_classes = np.argmax(Y_test, axis=1)

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


In [27]:
# Mostrar el reporte de clasificación
print(classification_report(y_test_classes, y_pred_classes, target_names=label_encoder.classes_))

              precision    recall  f1-score   support

  Adormilado       0.14      0.16      0.15        31
    Agobiado       0.00      0.00      0.00        22
  Agradecido       0.06      0.52      0.11        29
     Aislado       0.00      0.00      0.00        17
     Amoroso       0.00      0.00      0.00        18
 Avergonzado       0.00      0.00      0.00        34
      Celoso       0.05      0.55      0.10        22
    Confiado       0.00      0.00      0.00        22
    Distante       0.00      0.00      0.00        21
    Estupido       0.00      0.00      0.00        20
     Exitoso       0.00      0.00      0.00        38
   Frustrado       0.00      0.00      0.00        24
    Irritado       0.00      0.00      0.00        39
    Jugueton       0.00      0.00      0.00        21
   Optimista       0.00      0.00      0.00        22
       Osado       0.00      0.00      0.00        18
    Relajado       0.00      0.00      0.00        31
      Seguro       0.00    

c:\Users\WilmarAl\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\WilmarAl\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\WilmarAl\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av

# Pruebas

In [28]:
# Texto de entrada
user_text = 'La verdad extraño la persona que solías ser, que me recordaba que debo descansar sin sentirme culpable, que se molestó en conocer mi lenguaje corporal y me hizo sentir que quererme no era difícil. Te lo agradezco, pero te llevaste mi parte más vulnerable y ahora no sé qué hacer.'

# 1. Preprocesar el texto de entrada
user_tokens = preprocess_text(user_text)  # Tokenización
user_vector = text_to_vector(user_tokens, word2vec_model)  # Convertir a vector

# 2. Convertir el vector en un formato compatible con el modelo
user_vector = np.array([user_vector])  # Añadir una dimensión extra (batch size = 1)

# 3. Hacer la predicción con la red neuronal
prediction = model_ann.predict(user_vector)  # Obtener las probabilidades de cada clase
predicted_class_index = np.argmax(prediction, axis=1)  # Obtener la clase predicha

# 4. Convertir el índice de la clase predicha a la etiqueta original
predicted_class = label_encoder.inverse_transform(predicted_class_index)

# 5. Mostrar el resultado
print(f"Texto: {user_text}")
print(f"Clase predicha: {predicted_class[0]}")
print(f"Probabilidades por clase: {prediction}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Texto: La verdad extraño la persona que solías ser, que me recordaba que debo descansar sin sentirme culpable, que se molestó en conocer mi lenguaje corporal y me hizo sentir que quererme no era difícil. Te lo agradezco, pero te llevaste mi parte más vulnerable y ahora no sé qué hacer.
Clase predicha: Celoso
Probabilidades por clase: [[0.05962094 0.0564329  0.0606391  0.05060411 0.05685374 0.04160983
  0.06138128 0.05848372 0.04861556 0.03139533 0.05058502 0.03859854
  0.04753034 0.05280064 0.0418862  0.04467924 0.05242371 0.05649063
  0.04271631 0.04665283]]
